# Automatic Differentiation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChemAI-Lab/AI4Chem/blob/main/website/modules/03-automatic_differentiation.ipynb)
   

**References:**
1. **Automatic Differentiation in Machine Learning: a Survey** [Paper PDF](https://www.jmlr.org/papers/volume18/17-468/17-468.pdf)
1. **Chapters 5**: [Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf), C. M. Bishop.
<!-- 2. **Chapter 2**: [Machine Learning in Quantum Sciences](https://arxiv.org/pdf/2204.04198) UPDATE THIS! -->
3. **Chapter 7**: [Probabilistic Machine Learning: An Introduction, K. P. Murphy.](https://probml.github.io/pml-book/book1.html)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Introduction

Numerical differentiation is a computational field which "sole" aim is to estimate the derivate of a mathematical function or subroutine using values of the function. 
For example, one could use an approximation of the **derivative** to compute the gradient of a function,
$$
\frac{\partial f(\mathbf{x})}{\partial x_i} \approx \frac{f(\mathbf{x} - h\mathbf{e}_i) - f(\mathbf{x})}{h},
$$
where $\mathbf{e}_i$ is a unit vector in the $i$-th element, and $h$ is a small positive step size.  
From this equation one can observe two things, 
1. The estimation of the gradient is "simple", it only depends on the evaluation of the function itself. 
2. We require $2N$ total evaluations to ensemble the full jacobian, $\nabla f(\mathbf{x})$.

## 1D Example
$$
f(x) = \sin(x) + 0.1x^2
$$
where
$$
\frac{\partial f(x)}{\partial x} = \cos(x) + 0.2x
$$

In [ ]:
# -----------------------------
# 1D example: finite difference vs exact derivative
# f(x) = sin(x) + 0.1 x^2
# f'(x) = cos(x) + 0.2 x
# -----------------------------


def f(x):
    return np.sin(x) + 0.1 * x**2


def fp_exact(x):
    return np.cos(x) + 0.2 * x


def fp_forward_fd(x, h):
    return (f(x + h) - f(x)) / h


# Domain
x = np.linspace(-6, 6, 1200)
y_exact = fp_exact(x)

# Step sizes to animate through (log-spaced, typical slide-friendly range)
h_values = np.logspace(0, -8, 50)

# Precompute errors for y-axis scaling stability (optional but makes animation nicer)
all_err = []
all_approx = []
for h in h_values:
    ya = fp_forward_fd(x, h)
    all_approx.append(ya)
    all_err.append(np.abs(ya - y_exact))

all_approx = np.array(all_approx)
all_err = np.array(all_err)

# Figure: side-by-side panels
fig, (ax1, ax2) = plt.subplots(
    1, 2, figsize=(11, 4.2), constrained_layout=True)

# Panel 1: derivatives
ax1.set_title("Derivative: exact vs forward finite difference")
ax1.set_xlabel("x")
ax1.set_ylabel("f'(x)")
(line_exact,) = ax1.plot(x, y_exact, lw=2, label="Exact", alpha=0.85)
(line_fd,) = ax1.plot(x, all_approx[0], lw=2, label="Forward FD", alpha=0.9)
ax1.legend(loc=4)
ax1.grid(True, alpha=0.25)

# Set reasonable y-limits based on exact +/- margin
ymin = min(y_exact.min(), all_approx.min())
ymax = max(y_exact.max(), all_approx.max())
pad = 0.05 * (ymax - ymin + 1e-12)
ax1.set_ylim(ymin - pad, ymax + pad)

# Panel 2: absolute error
ax2.set_xlabel("x")
ax2.set_ylabel("Absolute error |FD - exact|")
(line_err,) = ax2.plot(x, all_err[0], lw=2)
ax2.set_yscale("log")  # log scale makes the tradeoff pop
ax2.grid(True, alpha=0.25)

# Stable y-limits for log plot (avoid zero)
err_min = np.maximum(all_err.min(), 1e-16)
err_max = np.maximum(all_err.max(), 1e-16)
ax2.set_ylim(err_min, err_max)

# Text annotation for current h
h_text = ax1.text(
    0.02, 0.95, "", transform=ax1.transAxes, va="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.85, edgecolor="0.8")
)


def init():
    line_fd.set_ydata(all_approx[0])
    line_err.set_ydata(all_err[0])
    h_text.set_text(f"h = {h_values[0]:.1e}")
    return line_fd, line_err, h_text


def update(frame):
    ya = all_approx[frame]
    ea = all_err[frame]

    line_fd.set_ydata(ya)
    line_err.set_ydata(np.maximum(ea, 1e-16))  # keep strictly positive for log
    h_text.set_text(f"h = {h_values[frame]:.1e}")

    return line_fd, line_err, h_text


ani_fd = animation.FuncAnimation(
    fig,
    update,
    frames=len(h_values),
    init_func=init,
    interval=120,   # adjust speed
    blit=False
)

plt.close(fig)   
HTML(ani_fd.to_jshtml())

## 2D Example
$$
f(x_1,x_2) = \sin(x_1)\cos(x_2) + 0.1(x_1^2 + x_2^2)
$$
where
$$
\nabla f(x_1,x_2) = \begin{pmatrix}
\frac{\partial f(x_1,x_2) }{\partial x_1} \\
\frac{\partial f(x_1,x_2) }{\partial x_2}
\end{pmatrix} =  \begin{pmatrix}
 \cos(x_1) \cos(x_2) + 0.2x_1 \\
 -\sin(x_1) \sin(x_2) + 0.2x_2 \\
\end{pmatrix}
$$

In [ ]:
# -----------------------------
# 2D example: finite difference gradient vs exact gradient
# f(x,y) = sin(x)cos(y) + 0.1(x^2 + y^2)
# -----------------------------


def f(x, y):
    return np.sin(x) * np.cos(y) + 0.1 * (x**2 + y**2)


def grad_exact(x, y):
    gx = np.cos(x) * np.cos(y) + 0.2 * x
    gy = -np.sin(x) * np.sin(y) + 0.2 * y
    return gx, gy


def grad_forward_fd(x, y, h):
    gx = (f(x + h, y) - f(x, y)) / h
    gy = (f(x, y + h) - f(x, y)) / h
    return gx, gy


# Grid (keep modest for animation performance)
x = np.linspace(-4, 4, 41)
y = np.linspace(-4, 4, 41)
X, Y = np.meshgrid(x, y)

Z = f(X, Y)
Gx_true, Gy_true = grad_exact(X, Y)

# Step sizes to animate (big -> tiny so you see truncation -> roundoff)
h_values = np.logspace(0, -5, 50)

# Precompute FD gradients + error magnitude for smooth animation
Gx_fd_all = []
Gy_fd_all = []
Err_all = []
for h in h_values:
    Gx_fd, Gy_fd = grad_forward_fd(X, Y, h)
    Gx_fd_all.append(Gx_fd)
    Gy_fd_all.append(Gy_fd)
    Err_all.append(np.sqrt((Gx_fd - Gx_true) ** 2 + (Gy_fd - Gy_true) ** 2))

Gx_fd_all = np.array(Gx_fd_all)
Gy_fd_all = np.array(Gy_fd_all)
Err_all = np.array(Err_all)

# Figure: 2x2 layout (nice for slides)
fig, axs = plt.subplots(2, 2, figsize=(12, 9), constrained_layout=True)
(ax_contour, ax_true), (ax_fd, ax_err) = axs

# --- Top-left: contours of f(x,y)
ax_contour.set_title("f(x, y) contours")
ax_contour.set_xlabel("x")
ax_contour.set_ylabel("y")
cs = ax_contour.contour(X, Y, Z, levels=18)
ax_contour.clabel(cs, inline=True, fontsize=8)

# --- Top-right: true gradient field
ax_true.set_title("True gradient ∇f(x, y)")
ax_true.set_xlabel("x")
ax_true.set_ylabel("y")
q_true = ax_true.quiver(X, Y, Gx_true, Gy_true)
ax_true.set_aspect("equal")

# --- Bottom-left: FD gradient field (animated)
ax_fd.set_title("Forward finite-difference gradient (animated)")
ax_fd.set_xlabel("x")
ax_fd.set_ylabel("y")
q_fd = ax_fd.quiver(X, Y, Gx_fd_all[0], Gy_fd_all[0])
ax_fd.set_aspect("equal")

# --- Bottom-right: error magnitude heatmap (animated)
ax_err.set_title("Error magnitude ‖∇f_FD − ∇f‖₂ (animated)")
ax_err.set_xlabel("x")
ax_err.set_ylabel("y")

# Use log scale on color via LogNorm-ish manual trick: plot log10(error)
# (avoids importing matplotlib.colors; keeps code minimal)
log_err0 = np.log10(np.maximum(Err_all[0], 1e-16))
im = ax_err.imshow(
    log_err0,
    origin="lower",
    extent=[x.min(), x.max(), y.min(), y.max()],
    aspect="equal"
)
cbar = fig.colorbar(im, ax=ax_err, shrink=0.85)
cbar.set_label("log10(error)")

# Annotation for h
h_text = ax_fd.text(
    0.02, 0.98, "", transform=ax_fd.transAxes, va="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.85, edgecolor="0.8")
)


def init():
    q_fd.set_UVC(Gx_fd_all[0], Gy_fd_all[0])
    im.set_data(np.log10(np.maximum(Err_all[0], 1e-16)))
    h_text.set_text(f"h = {h_values[0]:.1e}")
    return q_fd, im, h_text


def update(frame):
    q_fd.set_UVC(Gx_fd_all[frame], Gy_fd_all[frame])
    im.set_data(np.log10(np.maximum(Err_all[frame], 1e-16)))
    h_text.set_text(f"h = {h_values[frame]:.1e}")
    return q_fd, im, h_text


ani_2d = animation.FuncAnimation(
    fig, update, frames=len(h_values), init_func=init, interval=140, blit=False
)

plt.close(fig)
HTML(ani_2d.to_jshtml())

Various techniques have been developed to mitigate approximation errors in numerical differentiation, such as using a **center difference approximation**, 
$$
\frac{\partial f(\mathbf{x})}{\partial x_i} \approx \frac{f(\mathbf{x} + h\mathbf{e}_i) - f(\mathbf{x} - h\mathbf{e}_i)}{2h} + {\cal O}(h^2),
$$

# Forward Mode differentiation
AD in forward accumulation mode is how "physicist" see the chain rule. Let's consider the function $f(x_1,x_2) = \sin(x_1)\cos(x_2) + 0.1(x_1^2 + x_2^2)$, and compute its jacobian. First, we will associate each intermediate variable $v_i$ as $ \dot{v}_i = \frac{\partial v_i}{\partial x_j}$. Similar to finite differences, we will initialized with only one of the $\dot{v}_i$ variables being non-zero; e.g., $\dot{v}_i = \frac{\partial v_i}{\partial x_1} = 1$ and $\dot{v}_i = \frac{\partial v_i}{\partial x_2} = 0$.

| Forward Primal Trace      | Forward Tangent (Derivative) Trace|
| ----------- | ----------- |
| $v_{-1} = x_1$     | $\dot{v}_{-1} = \dot{x}_1$ = 1       |
| $v_{0} = x_2$   | $\dot{v}_{0} = \dot{x}_2$ = 0           |
| ----------- | ----------- |
| $v_{1} = \sin(x_1)$     | $\dot{v}_{1} = \cos(v_{-1})\dot{v}_{-1}$       |
| $v_{2} = \cos(x_0)$     | $\dot{v}_2 = -\sin(v_0)\dot{v}_0$       |
| $v_{3} = v_{-1}^2$     | $\dot{v}_3 = 2v_{-1}\dot{v}_{-1}$      |
| $v_{4} = v_{0}^2$     | $\dot{v}_4 = 2v_0\dot{v}_0$       |
| $v_{5} = v_{1} \times v_{2}$     | $\dot{v}_5 = \dot{v}_1 \, v_{2} + v_{1}\,\dot{v}_2$       |
| $v_{6} = v_{3} + v_{4}$     | $\dot{v}_6 = \dot{v}_3 + \dot{v}_4$       |
| $v_{7} = 0.1*v_{6}$     | $\dot{v}_7 = 0.1 \dot{v}_{6}$       |
| $v_{8} = v_{7} + v_{7}$     | $\dot{v}_2 = \dot{v}_5 + \dot{v}_7$       |
| ----------- | ----------- |
| $y = v_{8}$ |    $\dot{y} = \dot{v}_{8}$        |


Forward mode AD provides a very efficient and matrix-free way of computing Jacobian–vector products,
$$
\mathbf{J}_{f}\mathbf{r}  = \begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n}\\
\vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_n} \\
\end{bmatrix}\begin{bmatrix}
r_1\\
\vdots \\
r_n
\end{bmatrix}
$$
where $\dot{\mathbf{x}} = \mathbf{r}$. Thus, we can compute the Jacobian–vector product in
just one forward pass.

* Compared to finite differences, forward mode AD only requires $N$ evaluations of the function $f$ to compute the jacobian, one for each variable, and its the exact gradient.

In [ ]:

def forward_mode_trace_np(x1, x2, seed=(1.0, 0.0), verbose=True):
    """
    Forward-mode AD for:
        f(x1, x2) = sin(x1)*cos(x2) + 0.1*(x1^2 + x2^2)

    seed = (dx1, dx2):
        (1,0) -> ∂f/∂x1
        (0,1) -> ∂f/∂x2
        (a,b) -> directional derivative a*∂f/∂x1 + b*∂f/∂x2
    """
    dx1, dx2 = seed

    # --- Inputs (your v_-1 and v_0) ---
    v_m1, dv_m1 = np.array(x1, dtype=float), float(dx1)   # v_-1 = x1
    v_0,  dv_0 = np.array(x2, dtype=float), float(dx2)   # v_0  = x2

    # --- Primal + tangent trace ---
    v1 = np.sin(v_m1)
    dv1 = np.cos(v_m1) * dv_m1
    v2 = np.cos(v_0)
    dv2 = -np.sin(v_0) * dv_0

    v3 = v_m1**2
    dv3 = 2.0 * v_m1 * dv_m1
    v4 = v_0**2
    dv4 = 2.0 * v_0 * dv_0

    v5 = v1 * v2
    dv5 = dv1 * v2 + v1 * dv2
    v6 = v3 + v4
    dv6 = dv3 + dv4
    v7 = 0.1 * v6
    dv7 = 0.1 * dv6
    y = v5 + v7
    dy = dv5 + dv7

    if verbose:
        rows = [
            ("v_-1 = x1", v_m1, "dv_-1 = dx1", dv_m1),
            ("v_0  = x2", v_0,  "dv_0  = dx2", dv_0),
            ("v1 = sin(v_-1)", v1, "dv1 = cos(v_-1)*dv_-1", dv1),
            ("v2 = cos(v_0)",  v2, "dv2 = -sin(v_0)*dv_0",  dv2),
            ("v3 = v_-1^2",    v3, "dv3 = 2*v_-1*dv_-1",    dv3),
            ("v4 = v_0^2",     v4, "dv4 = 2*v_0*dv_0",      dv4),
            ("v5 = v1*v2",     v5, "dv5 = dv1*v2 + v1*dv2", dv5),
            ("v6 = v3+v4",     v6, "dv6 = dv3 + dv4",       dv6),
            ("v7 = 0.1*v6",    v7, "dv7 = 0.1*dv6",         dv7),
            ("y = v5+v7",      y,  "dy = dv5 + dv7",        dy),
        ]
        for a, av, b, bv in rows:
            print(f"{a:<18} = {av: .8f}   |   {b:<28} = {bv: .8f}")

    return float(y), float(dy)


# Example usage:
x1, x2 = 1.2, -0.7

# ∂f/∂x1 at (x1,x2)
y, dfdx1 = forward_mode_trace_np(x1, x2, seed=(1.0, 0.0), verbose=True)
print("\nForward-mode AD")
print("∂f/∂x1 =", dfdx1)

# ∂f/∂x2 at (x1,x2)
_, dfdx2 = forward_mode_trace_np(x1, x2, seed=(0.0, 1.0), verbose=False)
print("∂f/∂x2 =", dfdx2)

print("\nExact gradient:")
dfdx1_true, dfdx2_true = grad_exact(x1, x2)
print("∂f/∂x1 =", dfdx1_true)
print("∂f/∂x2 =", dfdx2_true)

# finite difference check
h_fd = 1e-2
dfdx1_fd, dfdx2_fd = grad_forward_fd(x1, x2, h_fd)
print("\nFinite difference gradient (h = 1e-6):")
print("∂f/∂x1 =", dfdx1_fd)
print("∂f/∂x2 =", dfdx2_fd)

## Forward AD as Dual Numbers 
Commonly, one does not "program" the derivative either use the **dual numbers** representation, $v + \dot{v}\epsilon$. 
$\epsilon$ is a **nilpotent number**, meaning $\epsilon^2 = 0$ and $\epsilon \neq 0$. <br>
$$
f(v + \dot{v}\epsilon) = f(v) + f'(v)\dot{v}\epsilon,
$$
where $f'(v)$ is the derivative of $f$ with respect to $v$. This will allows us to overload the operations and expand simple functions to also aggregate the gradient.

```Python
import numpy as np

class Dual:
    def __init__(self, val, dot):
        self.val = val      # primal
        self.dot = dot      # tangent

    def __add__(self, other):
        if not isinstance(other, Dual):
            other = Dual(other, 0.0)
        return Dual(self.val + other.val,
                    self.dot + other.dot)

    def __mul__(self, other):
        if not isinstance(other, Dual):
            other = Dual(other, 0.0)
        return Dual(self.val * other.val,
                    self.dot * other.val + self.val * other.dot)

    def __pow__(self, p):
        return Dual(self.val ** p,
                    p * self.val ** (p - 1) * self.dot)

# overload simple functions
def sin(x):
    return Dual(np.sin(x.val),
                np.cos(x.val) * x.dot)

def cos(x):
    return Dual(np.cos(x.val),
                -np.sin(x.val) * x.dot)
```

# Reverse Mode AD
Reverse AD corresponds to a generalized backpropagation algorithm as it propagates the derivatives from the outputs to the inputs.
This is done using intermediate variables $v_i$ named **adjoint**, 
$$
\bar{v}_i = \frac{\partial y_j}{\partial v_i}
$$
Derivatives are computed in the second phase of a two-phase process.
In the first phase, the original function code is run forward, populating intermediate variables vi and recording the dependencies in the computational graph through a book-keeping procedure. In the second phase, derivatives are calculated by propagating adjoints $\bar{v}_i$ in reverse, from the outputs to the inputs.

Reverse-mode AD is literally computing:
$$
\frac{\partial y}{\partial v_i} = \sum_{j\in\text{children(i)}} \frac{\partial y}{\partial v_j}\frac{\partial v_j}{\partial v_i}
$$



| Forward Primal Trace      | Reverse Adjoint (Derivative) Trace |
| ----------- | ----------- |
| $v_{-1} = x_1$     | $\bar{x}_1 = \bar{v}_{-1}$  |
| $v_{0} = x_2$   |  $\bar{x}_0 = \bar{v}_{0} $         |
| ----------- | ----------- |
| $v_{1} = \sin(v_{-1})$     | $\bar{v}_{-1} = \bar{v}_{-1} + \bar{v}_1\frac{\partial v_1}{\partial v_{-1}}$         |
| $v_{2} = \cos(v_0)$     | $\bar{v}_{0} = \bar{v}_{0} + \bar{v}_2\frac{\partial v_2}{\partial v_{0}}$       |
| $v_{3} = v_{-1}^2$     | $\bar{v}_{-1} = \bar{v}_3\frac{\partial v_3}{\partial v_{-1}}$     |
| $v_{4} = v_{0}^2$     | $\bar{v}_0 = \bar{v}_4\frac{\partial v_4}{\partial v_0}$      |
| $v_{5} = v_{1} \times v_{2}$     |  $\bar{v}_2 = \bar{v}_5\frac{\partial v_5}{\partial v_2}$   and $\bar{v}_1 = \bar{v}_5\frac{\partial v_5}{\partial v_1}$     |
| $v_{6} = v_{3} + v_{4}$     |  $\bar{v}_4 = \bar{v}_6\frac{\partial v_6}{\partial v_4}$   and $\bar{v}_3 = \bar{v}_6\frac{\partial v_6}{\partial v_3}$        |
| $v_{7} = 0.1*v_{6}$     |  $\bar{v}_6 = \bar{v}_6\frac{\partial v_7}{\partial v_6}$     |
| $v_{8} = v_{5} + v_{7}$     | $\bar{v}_7 = \bar{v}_8\frac{\partial v_8}{\partial v_7}$ and $\bar{v}_5 = \bar{v}_8\frac{\partial v_8}{\partial v_5}$       |
| ----------- | ----------- |
| $y = v_{8}$ |    $\dot{y} = \dot{v}_{8}$        |

Similar to the matrix-free computation of Jacobian–vector products with forward mode, reverse mode can be used for computing the transposed Jacobian–vector product
$$
\mathbf{J}^\top_{f}\mathbf{r}  = \begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_1}\\
\vdots & \ddots & \vdots \\
\frac{\partial y_1}{\partial x_n} & \cdots & \frac{\partial y_m}{\partial x_n} \\
\end{bmatrix}\begin{bmatrix}
r_1\\
\vdots \\
r_m
\end{bmatrix}
$$
by initializing the reverse phase with $\bar{\mathbf{y}} = \bar{\mathbf{r}}$.

* Reverse mode AD can build the full jacobian for a scalar function with a single forward pass, with the additional cost to store the adjoints in the way. (This is the reason why bigger GPUs are needed during training than inference).
